In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import functools
import math
import livelossplot
import pickle
from Utils.helper import *
from Utils.sample_squads import *

In [ ]:
model_inn_1 = tf.keras.models.load_model('Models/Inn1_SimpleRNN_10.h5')
# model_inn_2 = tf.keras.models.load_model('Models/Inn2_SimpleRNN_10.h5')

In [ ]:
inn1_df = pd.DataFrame(columns=BF_Cols)
inn1 = Innings(RCB_Squad[0], CSK_Squad[1], CSK_Squad[0][0], RCB_Pitch, 1, inn1_df)
inn1.simulate_inning(model_inn_1)
display_batting_table(inn1)

In [ ]:
inn1.inn_progress_df

In [ ]:
print("Simulating First Match")
match_sim2 = Match(CSK_Squad, MI_Squad, MI_Pitch, model_inn_1, model_inn_2, Display=1)